In [3]:
import pandas as pd
import pyodbc

In [43]:
conn = pyodbc.connect(
    "DRIVER={MySQL ODBC 8.0 Unicode Driver};"
    "SERVER=localhost;"
    "PORT=3306;"
    "DATABASE=sql_project_1;"
    "USER=root;"
    "PASSWORD=Mymummy1@;"
    "OPTION=3;"
)

In [44]:
query = "SELECT * FROM retail_sales"
basic = pd.read_sql(query, conn)

c:\Users\Admin\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [45]:
len(basic)

967

In [46]:
q2 = """ 
SELECT COUNT(*) FROM retail_sales;
"""

pd.read_sql(q2, conn)

c:\Users\Admin\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,COUNT(*)
0,967


In [47]:
q3 = """ 
SELECT * FROM retail_sales
WHERE age IS NULL OR
cogs IS NULL;
"""

pd.read_sql(q3, conn)

c:\Users\Admin\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,transactions_id,sale_date,sale_time,customer_id,gender,age,category,quantity,price_per_unit,cogs,total_sale
0,150,2022-04-13,08:25:00,89,Female,NaN,Electronics,4.0,30.0,16.20,120.0
1,432,2022-03-10,11:31:00,17,Female,NaN,Electronics,2.0,500.0,190.00,1000.0
2,679,2022-08-26,08:59:00,64,Female,18.0,Beauty,NaN,NaN,NaN,NaN
3,746,2022-07-05,11:33:00,42,Female,33.0,Clothing,NaN,NaN,NaN,NaN
4,797,2022-09-16,06:38:00,116,Male,NaN,Clothing,3.0,25.0,10.75,75.0
5,845,2022-10-27,10:12:00,25,Male,NaN,Clothing,1.0,500.0,145.00,500.0
6,921,2022-09-28,09:34:00,101,Male,NaN,Electronics,3.0,25.0,8.00,75.0
7,1150,2022-08-22,10:04:00,77,Female,NaN,Electronics,4.0,30.0,10.20,120.0
8,1225,2022-02-02,09:51:00,137,Female,57.0,Beauty,NaN,NaN,NaN,NaN
9,1367,2022-04-15,11:38:00,16,Female,NaN,Electronics,1.0,50.0,15.50,50.0


### Data Exploration

In [48]:
q4 = """ 
SELECT COUNT(DISTINCT customer_id) as unique_customers FROM retail_sales
"""

pd.read_sql(q4, conn)

c:\Users\Admin\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,unique_customers
0,149
